In [1]:
import torch
import torch.nn as nn
print(*torch.__config__.show().split('\n'),sep='\n')

PyTorch built with:
  - GCC 4.2
  - C++ Version: 201703
  - clang 13.1.6
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: NO AVX
  - Build settings: BLAS_INFO=accelerate, BUILD_TYPE=Release, CXX_COMPILER=/Applications/Xcode_13.3.1.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/c++, CXX_FLAGS= -Wno-deprecated -fvisibility-inlines-hidden -Wno-deprecated-declarations -DUSE_PTHREADPOOL -DNDEBUG -DUSE_KINETO -DLIBKINETO_NOCUPTI -DLIBKINETO_NOROCTRACER -DUSE_PYTORCH_QNNPACK -DUSE_XNNPACK -DUSE_PYTORCH_METAL_EXPORT -DSYMBOLICATE_MOBILE_DEBUG_HANDLE -DUSE_COREML_DELEGATE -O2 -fPIC -Wall -Wextra -Werror=return-type -Werror=non-virtual-dtor -Werror=braced-scalar-init -Werror=range-loop-construct -Werror=bool-operation -Winconsistent-missing-override -Wnarrowing -Wno-missing-field-initializers -Wno-type-limits -Wno-array-bounds -Wno-unknown-pragmas -Wunused-local-typedefs -Wno-unused-parameter -Wno-unused-function -Wno-unused-resu

In [4]:
from regex import X
import torch
import torch.nn as nn

x = torch.randn(4,2)
W = torch.randn(3,4)
print(x)
print(W)

tensor([[-0.7326,  1.9864],
        [ 0.2948, -0.6799],
        [ 1.5321, -0.8278],
        [-0.9015,  0.9813]])
tensor([[ 0.2428,  0.0112, -0.2120,  1.1052],
        [-0.0213, -0.9835,  0.2258,  0.3175],
        [ 1.1321,  0.7808, -1.9536,  0.3368]])


In [5]:
torch.matmul(W,x)

tensor([[-1.4958,  1.7348],
        [-0.2145,  0.7510],
        [-3.8962,  3.6657]])

In [7]:
import numpy as np
x = torch.Tensor([[2,3],[1,2]])
print(x)

x = torch.Tensor(np.array([[-1,1],[2,4]]))
print(x)

x = torch.zeros((2,3), dtype=torch.int32)
print(x)


tensor([[2., 3.],
        [1., 2.]])
tensor([[-1.,  1.],
        [ 2.,  4.]])
tensor([[0, 0, 0],
        [0, 0, 0]], dtype=torch.int32)


In [8]:
x = torch.randn(4,2)
z = torch.randn(4,2)

print(x)
print(z)

tensor([[ 0.6200,  0.6477],
        [-3.5445,  1.2991],
        [-1.0220,  0.4600],
        [ 0.5248,  0.5739]])
tensor([[-1.4571, -0.3928],
        [-0.8425, -0.8855],
        [-0.3203, -1.0144],
        [-0.2308, -1.2271]])


In [9]:
x * z

tensor([[-0.9035, -0.2544],
        [ 2.9861, -1.1504],
        [ 0.3274, -0.4666],
        [-0.1211, -0.7042]])

In [ ]:
class TryModel(BaseModel):
    def __init__(self, config):
        super(TryModel, self).__init__(config)
        self.define_model_parameters()
        self.init_model_parameters()

        if config.emb_file is not None:
            self.load_pretrained_embeddings(config.emb_file)
        
    def define_model_parameters(self):
        """
        Define the model's parameters here. e.g., embedding layers, etc.
        """
        raise NotImplementedError
    
    def init_model_parameters(self):
        """
        Initialize the model's parameters by uniform sampling from a range [-v, v].
        """
        raise NotImplementedError

In [ ]:
# x: 5-dimensional vector
# Model's parameters contain:
#   A: 5*5 matrix 
#   b: 5-dimensional vector
#   c: scalar

class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.define_model_parameters()
    
    def define_model_parameters(self):
        self.A = nn.Parameter(torch.randn(5,5))
        self.b = nn.Parameter(torch.randn(5))
        self.c = nn.Parameter(torch.randn(1))
    
    def forward(self, x):
        U = x.T
        print('U', U)
        M = torch.matmul(U, self.A)
        print('M', M)
        Mv = torch.matmul(M, x)
        print('Mv', Mv)
        bx = torch.matmul(self.b, x)
        print('bx', bx)
        y = Mv + bx + self.c
        return y    

x = torch.randn((5))
model = SimpleModel()
y = model(x)
print('y', y)

x tensor([ 0.3759,  0.2939, -1.3283,  0.8259,  1.1248])
U tensor([ 0.3759,  0.2939, -1.3283,  0.8259,  1.1248])
M tensor([-2.0198,  3.5372,  0.0694,  3.8226,  0.8834],
       grad_fn=<SqueezeBackward4>)
Mv tensor(4.3387, grad_fn=<DotBackward0>)
bx tensor(0.6802, grad_fn=<DotBackward0>)
y tensor([5.6281], grad_fn=<AddBackward0>)


In [15]:
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01)
loss_func = nn.MSELoss()

y_true = torch.FloatTensor([1.])
loss = loss_func(y, y_true)
print('loss', loss)

loss tensor(21.4196, grad_fn=<MseLossBackward0>)


In [ ]:
optimizer.zero_grad()
loss.backward(retain_graph=True)

for name, param in model.named_parameters():
    print(name, param.grad)

A tensor([[  1.3081,   1.0227,  -4.6222,   2.8737,   3.9138],
        [  1.0227,   0.7996,  -3.6138,   2.2467,   3.0600],
        [ -4.6222,  -3.6138,  16.3325, -10.1542, -13.8297],
        [  2.8737,   2.2467, -10.1542,   6.3131,   8.5981],
        [  3.9138,   3.0600, -13.8297,   8.5981,  11.7103]])
b tensor([  3.4797,   2.7205, -12.2954,   7.6443,  10.4112])
c tensor([9.2563])


In [33]:
# before gradient update
for name, param in model.named_parameters():
    print(name, param)

# gradient update
optimizer.step()
# After gradient update
for name, param in model.named_parameters():
    print(name, param)

A Parameter containing:
tensor([[-1.5993,  2.0890,  1.9761,  0.8898, -1.7362],
        [ 0.5825, -1.1184, -0.7337, -0.3273,  3.1011],
        [ 0.9182, -1.1797, -0.8027, -0.7802, -0.8727],
        [-0.9398, -0.2250, -0.1883,  0.9980,  0.5032],
        [ 0.1542,  1.3041, -1.0100,  1.3259, -1.0515]], requires_grad=True)
b Parameter containing:
tensor([ 0.8361, -0.9587, -0.1502,  0.0021,  0.1901], requires_grad=True)
c Parameter containing:
tensor([0.5504], requires_grad=True)
A Parameter containing:
tensor([[-1.6020,  2.0863,  1.9788,  0.8871, -1.7389],
        [ 0.5798, -1.1211, -0.7310, -0.3300,  3.0984],
        [ 0.9208, -1.1771, -0.8053, -0.7775, -0.8700],
        [-0.9424, -0.2276, -0.1856,  0.9953,  0.5005],
        [ 0.1515,  1.3014, -1.0073,  1.3232, -1.0542]], requires_grad=True)
b Parameter containing:
tensor([ 8.3345e-01, -9.6140e-01, -1.4755e-01, -5.5246e-04,  1.8741e-01],
       requires_grad=True)
c Parameter containing:
tensor([0.5477], requires_grad=True)
